### Importing of modules and packages

In [9]:
import pandas as pd
import matplotlib.pyplot as plt

### Assigning current 'Weekly-covid-admissions-and-bed-publication' to the variable 'current_covid_admissions' so data be updated weekly as new versions becomes available

In [10]:
current_covid_admissions ='/Users/ivan/Desktop/PycharmProjects/Data_Science_Mini/Data/raw/Weekly-covid-admissions-and-beds-publication-201224'

### Extracting last date from 'All beds COVID' sheet (any sheet could have been used) and assigning it to variable last_day

In [11]:
df = pd.read_excel(r'%s.xlsx' % current_covid_admissions,
                   sheet_name='All beds COVID',
                   engine='openpyxl')
last_index = df.iloc[13].last_valid_index()

# convert datetime to date in format
last_day = df.iloc[13][last_index].strftime('%d/%m/%Y')

## Function to read in summary data from 'All beds COVID' and 'MV beds COVID' sheets (7 rows from lines 18 to 24 of Excel sheet). Also converts headings to dates

In [14]:
list_of_regions = ['East of England','London', 'Midlands','North East and Yorkshire',
                   'North West', 'South East', 'South West','ENGLAND']
columns_to_drop = ['Type 1 Acute?', 'NHS England Region', 'Code']
datelist = pd.date_range(start='08/01/2020', end=last_day).strftime('%d-%m-%Y')


def new_summary_sheet(sheet_name):
    dataframe = sheet_name
    dataframe = pd.read_excel(r'%s.xlsx' % current_covid_admissions,
                            sheet_name=sheet_name,
                            header=14,
                            engine='openpyxl')
    dataframe = dataframe.loc[dataframe['Name'].isin(list_of_regions)]
    dataframe = dataframe.drop(columns=columns_to_drop)
    dataframe = dataframe.reset_index(drop=True)

    i = 0
    for col in dataframe.columns[1:len(datelist)+2]:
        dataframe.rename(columns={col:datelist[i]}, inplace=True)
        i += 1
    return dataframe


df_all_beds = new_summary_sheet('All beds COVID')

# Save all_beds_df to analysis
all_beds_df = df_all_beds
all_beds_df.to_csv('Data/Processed/all_beds_df.csv')

# allocate max number of all beds in England to variable max_all_beds
max_all_beds = df_all_beds.max(axis=1)[0]
# round up to the next 1000
max_all_beds = round(max_all_beds,-3) + 1000

# allocate max number of all beds across all regions to max_region_all_beds
# ignore first entry as this is a total for whole of England
max_region_all_beds = df_all_beds.max(axis=1)[1:].max()
# round up to the next 100
max_region_all_beds = round(max_region_all_beds, -2) + 100

df_MV = new_summary_sheet('MV beds COVID')

# save MV_df for analysis
MV_df = df_MV
MV_df.to_csv('Data/Processed/MV_df.csv')

# Descriptive Analysis


In [ ]:
df_MV.describe().round(1)


### Comparison of proportion of beds occupied by mechanical ventilation and all beds on user defined days

In [ ]:
# setting up new dataframe
def mv_vs_all_beds(date_to_compare):
    summary_df = pd.DataFrame()

    # assigning regions and values to summary_df
    summary_df['Region'] = df_MV['Name']
    summary_df['Mechanical Ventilation (MV)'] = df_MV[date_to_compare]
    summary_df['All Beds'] = df_all_beds[date_to_compare]
    summary_df['Non MV Beds'] = summary_df['All Beds'] - summary_df['Mechanical Ventilation (MV)']
    summary_df['Percentage on MV'] =((summary_df['Mechanical Ventilation (MV)'] /summary_df['All Beds']) * 100).round(1)


    # Add title to summary_df
    fig = plt.figure(figsize = (10,2))
    ax = fig.add_subplot(111)
    summary_df = summary_df.set_index(['Region']).reindex(['East of England','London', 'Midlands','North East and Yorkshire',
                                 'North West', 'South East', 'South West','ENGLAND']).reset_index()

    # make Region the index column 12
    summary_df.set_index('Region', inplace=True)

    # Rename last row to 'TOTAL'
    summary_df = summary_df.rename(index={'ENGLAND':"TOTAL"})

    # create a table of data for each date chosen
    ax.table(cellText = summary_df.values,
              rowLabels = summary_df.index,
              colLabels = summary_df.columns,
              loc = "center"
             )
    ax.set_title(f"Percentage of MV beds on {date_to_compare}")
    ax.axis("off")

    return summary_df

def make_bar_graph(summary_df):
    # plot bar chart of TOTAL row
    total_df = summary_df.copy()
    total_df.drop(total_df.index[0:7], inplace=True)
    # upper limit has been assigned to max_all_beds
    total_df[['Mechanical Ventilation (MV)','Non MV Beds']].plot(kind='bar', stacked=True, ylim=(0,max_all_beds))

    # clean up data ready for construction of bar chart
    no_totals = summary_df.copy()
    no_totals.drop('TOTAL', inplace=True)

    # set ylim so scales are the same on multiple graphs
    no_totals[['Mechanical Ventilation (MV)','Non MV Beds']].plot(kind='bar', stacked=True, ylim=(0,max_region_all_beds))

def comparison_dates(no_of_dates):
    for i in range(no_of_dates):
        date_to_compare = input(f"Please enter the {i+1} dates you would like to compare (dd-mm-yyyy)")
        make_bar_graph(mv_vs_all_beds(date_to_compare))

# save summary_df for analysis in notebook 'Descriptive Analysis' on 3rd December
third_dec_summary = mv_vs_all_beds('03-12-2020')
third_dec_summary.to_csv('Data/Processed/3rd_Dec.csv')

no_of_dates_to_compare = int(input("Please enter the number of dates you'd like to compare: "))
comparison_dates(no_of_dates_to_compare)



In [ ]:
# print(last_day)
# df[last_day].agg(['max','sum', 'mean','median','std']).round(1)


### Scatter plot of all beds vs MV beds (offset MV beds from 1 to 7 days later)

In [ ]:
#extract total row from both 'All beds COVID' and 'MV beds COVID'
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

# initialise variable to hold the smallest RMSE over the 6 days and assume that occurs
# at day 0
smallest_RMSE = 1000
day = 1
xlsx = pd.ExcelFile('%s.xlsx' % current_covid_admissions, engine='openpyxl')


for i in range(1,10):

    all_MV_beds = pd.read_excel(xlsx, 'MV beds COVID')
    all_beds_england = pd.read_excel(xlsx, 'All beds COVID')

    all_MV_beds = all_MV_beds.iloc[14][3+i:-1]
    all_beds_england = all_beds_england.iloc[14][4:-i]


    # need to reset the index here or NaNs will be introduced
    all_beds_vs_MV_beds = pd.concat([
        all_beds_england.reset_index(drop=True),
        all_MV_beds.reset_index(drop=True)],
        axis=1)
    all_beds_vs_MV_beds.columns = ['All Beds','MV Beds']

    # Linear Regression
    X = all_beds_vs_MV_beds.iloc[:,:-1]
    y = all_beds_vs_MV_beds.iloc[:,-1]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


    # create an instance of the LinearRegression class
    ln1 = LinearRegression()

    # Train the regression model
    ln1.fit(X_train, y_train)

    # print the coefficients
    print(f"{i} days difference")
    print(f"The intercept: {ln1.intercept_}")
    print(f"The trained coefficient is: {ln1.coef_}")

    # predicting the test set
    y_pred = ln1.predict(X_test)

    # RSME (with squared=False)
    RMSE = metrics.mean_squared_error(y_pred, y_test, squared=False)
    print(f"RMSE = {RMSE}")
    if RMSE < smallest_RMSE:
        smallest_RMSE = RMSE
        day = i

    print()

    # Visualising the Training set results
    plt.scatter(X_train, y_train, color='green')
    plt.plot(X_train, ln1.predict(X_train), color='yellow')
    plt.title(f"All beds vs MV beds (Training set) for day {i}")
    plt.xlabel('All beds')
    plt.ylabel('MV beds')
    plt.show()

    # Visualising the Test set results
    plt.scatter(X_test, y_test, color='red')

    # don't replace anything here as regression line is based on testing set
    plt.plot(X_train, ln1.predict(X_train), color='pink')
    plt.title(f"All beds vs MV beds (Test set) {i} days difference")
    plt.xlabel('All beds')
    plt.ylabel('MV beds')
    plt.show()



In [ ]:
print(f"The smallest RMSE is {smallest_RMSE.round(2)} from {day} day difference")

### Does the linear regression shows that after 1 day patients move onto a MV bed?

### Multiple Linear Regression
##### Independent variables 'All Beds' and 'Hospital Admissions and Diagnosis'
##### Dependent variable 'MV Beds'